# Solve a System of Equations, in Java
We will define a couple of classes and utilities, and finally solve a system of 3 equations with 3 unknown variables. The dimension of the system (3 here) is obviously *not* hard-coded.

The classes we need will be:
- `SquareMatrix`
- `MatrixUtil`
    - also defines a `MatrixException`, usable for example when a Matrix cannot be properly inverted.
- `SystemUtil`

It is not mandatory to define them, but is is certainly much clearer (and cleaner).

First we import some required classes.

In [1]:
import java.text.DecimalFormat;
import java.util.Arrays;
import java.util.List;
import java.util.stream.IntStream;

Next we define a class named `SquareMatrix`, it will hold the data of the System into a `double[][]`, and it has methods to manipulate its elements.

In [2]:
public final class SquareMatrix {
  private final int dimension;
  private double[][] matrixElements;

  public SquareMatrix(int dim) {
    this(dim, false);
  }

  public SquareMatrix(int dim, boolean init) {
    if (dim < 1) {
      throw new IllegalArgumentException("Dimension must be at least 1");
    }
    this.dimension = dim;
    matrixElements = new double[dim][dim];
    if (init) {
      for (int l = 0; l < dim; l++) {
        for (int c = 0; c < dim; c++)
          matrixElements[l][c] = 0d;
      }
    }
  }

  public SquareMatrix(int dim, double... elements) {
    this(dim);
    if (elements == null) {
      throw new IllegalArgumentException("Elements array cannot be null");
    }
    if (elements.length != (dim * dim)) {
      throw new IllegalArgumentException(String.format("Invalid number of elements for a matrix of dim %d, expecting %d, got %d", dim, (dim * dim), elements.length));
    }
    for (int l = 0; l < dim; l++) {
      for (int c = 0; c < dim; c++)
        matrixElements[l][c] = elements[(l * dim) + c];
    }
  }

  public static SquareMatrix builder(int dim) {
    return new SquareMatrix(dim);
  }

  public int getDimension() {
    return (this.dimension);
  }

  public void setElementAt(int row, int col, double val) {
    matrixElements[row][col] = val;
  }

  public double getElementAt(int row, int col) {
    return matrixElements[row][col];
  }

  public double[][] getMatrixElements() {
    return this.matrixElements;
  }

  public void setMatrixElements(double[][] me) {
    this.matrixElements = me;
  }
}


Here is a utility class (notice that most - if not all - of its members are `static`), defining operations on a square matrix, like
- `printMatrix`
- `minor`
- `comatrix`
- `transpoose`
- `multiply`
- `equals`
- `determinant`
- `invert`

As we all know, an inverted matrix is the transposed of its comatrix multiplied by the inverse of its determinant (which thus must be non null. If the determinant is null, the matrix simply cannot be inverted).
- A transposed matrix is a matrix where lines of the original replace its columns, and vice-versa.
- The comatrix is the original matrix where each term is replaced with the determinant of its minor.
- The minor of a matrix for the element `[x, y]` is the original matrix, without its column `x` and its line `y`.
- The `determinant` involves a recursive operation. See the code for details.

In [3]:
 public final class MatrixUtil {

  private static boolean debug = "true".equals(System.getProperty("debug", "false"));

  public static void printMatrix(SquareMatrix m) {
    printMatrix(m, true);
  }

  public static void printMatrix(SquareMatrix m, boolean withCR) {
    for (int row=0; row<m.getDimension(); row++) {
      String line = "| ";
      for (int col=0; col<m.getDimension(); col++) {
        line += (m.getElementAt(row, col) + " ");
      }
      line += " |";
      if (!withCR && row == (m.getDimension() - 1)) { // Last line
        System.out.print(line);
      } else {
        System.out.println(line);
      }
    }
  }

  private static SquareMatrix minor(SquareMatrix m, int row, int col) {
    SquareMatrix small = new SquareMatrix(m.getDimension() - 1);
    for (int c = 0; c < m.getDimension(); c++) {
      if (c != col) {
        for (int r = 0; r < m.getDimension(); r++) {
          if (r != row) {
            small.setElementAt(((r < row) ? r : (r - 1)), ((c < col) ? c : (c - 1)), m.getElementAt(r, c));
          }
        }
      }
    }
    return small;
  }

  public static SquareMatrix comatrix(SquareMatrix m) {
    SquareMatrix co = new SquareMatrix(m.getDimension());
    for (int r = 0; r < m.getDimension(); r++) {
      for (int c = 0; c < m.getDimension(); c++) {
        co.setElementAt(r, c, determinant(minor(m, r, c)) * Math.pow((-1), (r + c + 2)));  // r+c+2 = (r+1) + (c+1)...
      }
    }
    if (debug) {
      System.out.println("Comatrix:");
      printMatrix(co);
    }
    return co;
  }

  public static SquareMatrix transposed(SquareMatrix m) {
    SquareMatrix t = new SquareMatrix(m.getDimension());
    // Replace line with columns.
    int r, c;
    for (r = 0; r < m.getDimension(); r++) {
      for (c = 0; c < m.getDimension(); c++) {
        t.setElementAt(r, c, m.getElementAt(c, r));
      }
    }
    if (debug) {
      System.out.println("Transposed:");
      printMatrix(t);
    }
    return t;
  }

  public static SquareMatrix multiply(SquareMatrix m, double n) {
    SquareMatrix res = new SquareMatrix(m.getDimension());
    int r, c;

    for (r = 0; r < m.getDimension(); r++) {
      for (c = 0; c < m.getDimension(); c++) {
        res.setElementAt(r, c, m.getElementAt(r, c) * n);
      }
    }
    return res;
  }

  public static boolean equals(SquareMatrix a, SquareMatrix b) {
    if (a.getDimension() != b.getDimension()) {
      return false;
    }
    for (int r=0; r<a.getDimension(); r++) {
      for (int c=0; c<a.getDimension(); c++) {
        if (a.getElementAt(r, c) != b.getElementAt(r, c)) {
          return false;
        }
      }
    }
    return true;
  }

  public static double determinant(SquareMatrix m) {
    double v = 0.0;

    if (m.getDimension() == 1) {
      v = m.getElementAt(0, 0);
    } else {
      // C : column in Major
      for (int C = 0; C < m.getDimension(); C++) { // Walk thru first line
        // Minor's determinant
        double minDet = determinant(minor(m, 0, C));
        v += (m.getElementAt(0, C) * minDet * Math.pow((-1.0), C + 1 + 1)); // line C, column 1
      }
    }
    if (debug) {
      System.out.println("Determinant of");
      printMatrix(m, false);
      System.out.println(String.format(" is %f", v));
    }
    return v;
  }

  public static SquareMatrix invert(SquareMatrix m) throws MatrixException {
    double determinant = determinant(m);
    if (determinant == 0) {
      throw new MatrixException(String.format("Matrix cannot be inverted, its determinant is null."));
    }
    return multiply(transposed(comatrix(m)), (1.0 / determinant));
  }
     
  public static class MatrixException extends Exception {
    public MatrixException(String message) {
      super(message);
    }
  }
}


Yet another utility class, invoking the above, to present the user with an easy interface to solve a system of equations.

In [5]:
public class SystemUtil {

  public static double[] solveSystem(double[] m,
                                     double[] c) 
      throws MatrixUtil.MatrixException {
    SquareMatrix sma = new SquareMatrix(c.length);

    for (int i = 0; i < c.length; i++) {
      for (int j = 0; j < c.length; j++) {
        sma.setElementAt(i, j, m[(c.length * i) + j]);
      }
    }
    return solveSystem(sma, c);
  }

  /**
   * Solves a system, n equations, n unknowns.
   * <p>
   * the values we look for are x, y, z.
   * <pre>
   * ax + by + cz = X
   * Ax + By + Cz = Y
   * Px + Qy + Rz = Z
   * </pre>
   * @param m Coeffs matrix, n x n (left) from the system above
   * <pre>
   * | a b c |
   * | A B C |
   * | P Q R |
   * </pre>
   * @param c Constants array, n (right) <code>[X, Y, Z]</code> from the system above
   * @return the unknown array, n. <code>[x, y, z]</code> from the system above
   */
  public static double[] solveSystem(SquareMatrix m,
                                     double[] c) 
      throws MatrixUtil.MatrixException {
    double[] result;
    result = new double[m.getDimension()];

    SquareMatrix inv = MatrixUtil.invert(m);

    // Print inverted Matrix
    if ("true".equals(System.getProperty("debug", "false"))) {
      System.out.println("Inverted:");
      MatrixUtil.printMatrix(inv);
    }

    // Lines * Column
    for (int row = 0; row < m.getDimension(); row++) {
      result[row] = 0.0;
      for (int col = 0; col < m.getDimension(); col++) {
        result[row] += (inv.getElementAt(row, col) * c[col]);
      }
    }
    return result;
  }

  public static String formatSystem(SquareMatrix squareMatrix, double[] constants) {
    StringBuffer content = new StringBuffer();
    String unknowns = "ABCDEFGHIJKLMNOPQRSTUVWXYZ";
    // Nice IntStream s !
    IntStream.range(0, squareMatrix.getDimension()).forEach(row -> {
      final StringBuffer sb = new StringBuffer();
      IntStream.range(0, squareMatrix.getDimension()).forEach(col -> {
        sb.append(String.format("%s(%f x %c)", (!sb.toString().trim().isEmpty() ? " + " : ""), squareMatrix.getElementAt(row, col), unknowns.charAt(col)));
      });
      sb.append(String.format(" = %f", constants[row]));
      content.append(String.format("%s%s", sb.toString(), "\n"));
    });
    return content.toString().trim();
  }
    
  public static void printSystem(SquareMatrix squareMatrix, double[] constants) {
    System.out.println(formatSystem(squareMatrix, constants));
  }
  
}

    

### Let's go
Finally, all utilities and classes are dewfined, we can proceed.

We will be solving the following system:
```
    12 x A    +  13 x B +    14 x C = 234
    1.345 x A - 654 x B + 0.001 x C = 98.87
    23.09 x A + 5.3 x B - 12.34 x C = 9.876
```
We are looking for the values A, B, and C

Notice the actual call to the system resolutio, it is all in
```java
double[] result = SystemUtil.solveSystem(squareMatrix, constants);
```

In [6]:
SquareMatrix squareMatrix = new SquareMatrix(3);
/*
  Resolution of:
    12x    +  13y +    14z = 234
    1.345x - 654y + 0.001z = 98.87
    23.09x + 5.3y - 12.34z = 9.876
 */
squareMatrix.setElementAt(0, 0, 12);
squareMatrix.setElementAt(0, 1, 13);
squareMatrix.setElementAt(0, 2, 14);

squareMatrix.setElementAt(1, 0, 1.345);
squareMatrix.setElementAt(1, 1, -654);
squareMatrix.setElementAt(1, 2, 0.001);

squareMatrix.setElementAt(2, 0, 23.09);
squareMatrix.setElementAt(2, 1, 5.3);
squareMatrix.setElementAt(2, 2, -12.34);

double[] constants = new double[]{234, 98.87, 9.876};

System.out.println(String.format("Matrix Determinant: %f", MatrixUtil.determinant(squareMatrix)));

Matrix Determinant: 308572.160470


Now we know that the determinant of our matrix is not null, we can proceed

In [7]:
System.out.println("Solving:");
SystemUtil.printSystem(squareMatrix, constants);

long before = System.nanoTime();
double[] result = SystemUtil.solveSystem(squareMatrix, constants);
long after = System.nanoTime();
System.out.println(String.format("\nDone is %s \u212bs (nano-sec).", DecimalFormat.getInstance().format(after - before)));

System.out.println(String.format("A = %f", result[0]));
System.out.println(String.format("B = %f", result[1]));
System.out.println(String.format("C = %f", result[2]));

Solving:
(12.000000 x A) + (13.000000 x B) + (14.000000 x C) = 234.000000
(1.345000 x A) + (-654.000000 x B) + (0.001000 x C) = 98.870000
(23.090000 x A) + (5.300000 x B) + (-12.340000 x C) = 9.876000

Done is 528,628,778 Ås (nano-sec).
A = 6.488222
B = -0.137817
C = 11.280925


We found a solution, let's make sure it is right, let's see if we find the original constants when ujsing the solution we found:

In [8]:
// Proof:
double X = (squareMatrix.getElementAt(0, 0) * result[0]) + (squareMatrix.getElementAt(0, 1) * result[1]) + (squareMatrix.getElementAt(0, 2) * result[2]);
System.out.println(String.format("X: %f", X));
double Y = (squareMatrix.getElementAt(1, 0) * result[0]) + (squareMatrix.getElementAt(1, 1) * result[1]) + (squareMatrix.getElementAt(1, 2) * result[2]);
System.out.println(String.format("Y: %f", Y));
double Z = (squareMatrix.getElementAt(2, 0) * result[0]) + (squareMatrix.getElementAt(2, 1) * result[1]) + (squareMatrix.getElementAt(2, 2) * result[2]);
System.out.println(String.format("Z: %f", Z));

X: 234.000000
Y: 98.870000
Z: 9.876000


### Using the other `SquareMatrix` constructor
The `SquareMatrix` class has a constructor taking the whole matrix data as a `double[]` (dim 1).
Let's see if it work the same:

In [9]:
System.out.println("--- With one-line SquareMatrix constructor---");
// Using another SquareMatrix constructor
squareMatrix = new SquareMatrix(3, 12, 13, 14, 1.345, -654, 0.001, 23.09, 5.3, -12.34);
System.out.println("Solving:");
SystemUtil.printSystem(squareMatrix, constants);

before = System.nanoTime();
result = SystemUtil.solveSystem(squareMatrix, constants);
after = System.nanoTime();
System.out.println(String.format("\nDone in %s \u212bs (nano-sec).", DecimalFormat.getInstance().format(after - before)));

System.out.println(String.format("A = %f", result[0]));
System.out.println(String.format("B = %f", result[1]));
System.out.println(String.format("C = %f", result[2]));

--- With one-line SquareMatrix constructor---
Solving:
(12.000000 x A) + (13.000000 x B) + (14.000000 x C) = 234.000000
(1.345000 x A) + (-654.000000 x B) + (0.001000 x C) = 98.870000
(23.090000 x A) + (5.300000 x B) + (-12.340000 x C) = 9.876000

Done in 177,555,884 Ås (nano-sec).
A = 6.488222
B = -0.137817
C = 11.280925


Looks like this is the same result as above, good!

### What happens when the matrix cannot be inverted

In [17]:
squareMatrix = new SquareMatrix(3, 0, 0, 0, 0, 1, 0, 0, 0, 1);
System.out.println("Solving:");
SystemUtil.printSystem(squareMatrix, constants);

try {
  before = System.nanoTime();
  result = SystemUtil.solveSystem(squareMatrix, constants);
  after = System.nanoTime();
  System.out.println(String.format("\nDone in %s \u212bs (nano-sec).", DecimalFormat.getInstance().format(after - before)));
  System.out.println(String.format("A = %f", result[0]));
  System.out.println(String.format("B = %f", result[1]));
  System.out.println(String.format("C = %f", result[2]));
} catch (MatrixUtil.MatrixException exception) {
  // exception.printStackTrace();
  // System.err.println("====================");
  System.err.println(String.format("Problem solving system:\n%s\n%s", 
                                   SystemUtil.formatSystem(squareMatrix, constants), 
                                   exception.getMessage()));
} catch (Exception exception) {
  System.err.println("What???");
  exception.printStackTrace();
}



Solving:
(0.000000 x A) + (0.000000 x B) + (0.000000 x C) = 234.000000
(0.000000 x A) + (1.000000 x B) + (0.000000 x C) = 98.870000
(0.000000 x A) + (0.000000 x B) + (1.000000 x C) = 9.876000


Problem solving system:
(0.000000 x A) + (0.000000 x B) + (0.000000 x C) = 234.000000
(0.000000 x A) + (1.000000 x B) + (0.000000 x C) = 98.870000
(0.000000 x A) + (0.000000 x B) + (1.000000 x C) = 9.876000
Matrix cannot be inverted, its determinant is null.
